I was using this to test out my cosine similarity recommendation function. This is now used in app.py.

In [3]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [4]:
pkl_file = open('../data/df.pkl', 'rb')
df = pickle.load(pkl_file)
pkl_file.close() 

In [11]:
features = [
            'top_elev_(ft)', 
            'bottom_elev_(ft)', 
            'vert_rise_(ft)', 
            'slope_length_(ft)', 
            'avg_width_(ft)', 
            'slope_area_(acres)', 
            'avg_grade_(%)', 
            'max_grade_(%)', 
            'groomed']

X = df[features].values

In [12]:
X

array([[  1.19240000e+04,   1.08580000e+04,   1.06600000e+03, ...,
          5.00000000e+01,   6.90000000e+01,   0.00000000e+00],
       [  1.14050000e+04,   1.10050000e+04,   4.00000000e+02, ...,
          5.50000000e+01,   6.60000000e+01,   0.00000000e+00],
       [  1.18400000e+04,   1.14050000e+04,   4.35000000e+02, ...,
          4.20000000e+01,   7.60000000e+01,   0.00000000e+00],
       ..., 
       [  8.11000000e+03,   8.10000000e+03,   1.00000000e+01, ...,
          8.00000000e+00,   1.10000000e+01,   1.00000000e+00],
       [  8.11300000e+03,   8.10000000e+03,   1.30000000e+01, ...,
          1.20000000e+01,   1.20000000e+01,   1.00000000e+00],
       [  8.14700000e+03,   8.10000000e+03,   4.70000000e+01, ...,
          1.50000000e+01,   1.20000000e+01,   1.00000000e+00]])

In [13]:
def cos_sim_recommendations(trail_name, resort_name, X, n=5, resort=None):
    index = df.index[(df['trail_name'] == trail_name) & (df['resort'] == resort_name)][0]
    trail = X[index].reshape(1,-1)
    cs = cosine_similarity(trail, X)
    rec_index = np.argsort(cs)[0][::-1][1:]
    ordered_df = df.loc[rec_index]
    if resort:
        ordered_df = ordered_df[ordered_df['resort'] == resort]
    rec_df = ordered_df.head(n)
    orig_row = df.loc[[index]].rename(lambda x: 'original')
    total = pd.concat((orig_row,rec_df))
    return total

In [14]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [15]:
X

array([[ 1.12885212,  0.67904826,  1.5011197 , ...,  1.7960483 ,
         1.3829293 , -0.81952402],
       [ 0.72015105,  0.79497328, -0.24838454, ...,  2.18280168,
         1.22823665, -0.81952402],
       [ 1.06270397,  1.1104155 , -0.15644362, ...,  1.1772429 ,
         1.74387879, -0.81952402],
       ..., 
       [-1.87458885, -1.49592585, -1.27286899, ..., -1.45268006,
        -1.6077951 ,  1.22022049],
       [-1.87222642, -1.49592585, -1.26498834, ..., -1.14327736,
        -1.55623089,  1.22022049],
       [-1.84545217, -1.49592585, -1.17567432, ..., -0.91122533,
        -1.55623089,  1.22022049]])

In [360]:
cos_sim_recommendations('Sorensen Park','Winter Park',X,n=5)

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
original,Sorensen Park,9150.0,9090.0,60.0,1067.43,114.0,4.07,6.0,8.0,Beginner,Winter Park,CO,1,green,1,1
821,School Yard,6799.0,6732.0,66.0,980.00,129.0,2.90,7.0,9.0,Beginner,Diamond Peak,NV,1,green,1,1
673,Bubba’s Shortcut Upper,10335.0,10238.0,97.0,1200.00,118.0,3.30,8.0,11.0,Novice,Crested Butte,CO,1,green,2,1
932,Shoo Fly,9580.0,9450.0,130.0,1258.66,112.0,5.88,10.0,14.0,Beginner,Winter Park,CO,1,green,1,1
314,Little Hawk TRV,9369.0,9348.0,21.0,821.00,67.0,1.30,3.0,8.0,Beginner,Eldora,CO,1,green,1,1
373,Loop Road,7522.0,7381.0,141.0,1395.00,135.0,4.30,10.0,15.0,Intermediate,Alpine Meadows,CA,1,blue,4,2


In [366]:
'''least similar'''
cs = cosine_similarity(X[0].reshape(1,-1), X)[0]
css = list(enumerate(cs))
srtd = sorted(css, key=lambda x: x[1])[::-1]
srtd[-5:]

[(673, -0.95435945246191645),
 (821, -0.95588131662771347),
 (848, -0.96046174527542427),
 (314, -0.96122201033982335),
 (403, -0.96942497514661874)]

In [365]:
df.iloc[[0,673,821,848,314,403]]

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
0,Over the Rainbow,11924.0,10858.0,1066.0,2404.00,319.0,17.60,50.0,69.0,Expert,Loveland,CO,0,bb,6,4
673,Bubba’s Shortcut Upper,10335.0,10238.0,97.0,1200.00,118.0,3.30,8.0,11.0,Novice,Crested Butte,CO,1,green,2,1
821,School Yard,6799.0,6732.0,66.0,980.00,129.0,2.90,7.0,9.0,Beginner,Diamond Peak,NV,1,green,1,1
848,Sorensen Park,9150.0,9090.0,60.0,1067.43,114.0,4.07,6.0,8.0,Beginner,Winter Park,CO,1,green,1,1
314,Little Hawk TRV,9369.0,9348.0,21.0,821.00,67.0,1.30,3.0,8.0,Beginner,Eldora,CO,1,green,1,1
403,Lakeview to Weasel,7538.0,7483.0,55.0,1282.00,106.0,3.10,4.0,18.0,Novice,Alpine Meadows,CA,1,blue,2,2


In [356]:
df.describe()

,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),groomed,ability_nums,color_nums
count,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000
mean,10490.496889,9996.925333,494.554844,2217.169636,245.223387,11.983182,26.780444,42.180444,0.401778,4.182222,2.269333
std,1270.441564,1268.625031,380.848577,2261.595747,271.702734,17.708566,12.933885,19.401920,0.490476,1.501377,0.891534
min,6799.000000,6732.000000,5.000000,72.000000,3.000000,0.100000,2.000000,5.000000,0.000000,1.000000,1.000000
25%,9824.000000,9390.000000,227.000000,960.100000,112.000000,3.000000,17.000000,28.000000,0.000000,3.000000,2.000000
50%,10772.000000,10127.000000,400.000000,1658.600000,170.000000,6.500000,26.000000,42.000000,0.000000,4.000000,2.000000
75%,11433.000000,10979.000000,653.000000,2686.000000,273.000000,14.300000,35.000000,54.000000,1.000000,6.000000,3.000000
max,12716.000000,12227.000000,3293.000000,30653.000000,2596.000000,238.300000,78.000000,236.000000,1.000000,6.000000,4.000000


## adding a color filter

In [377]:
color = ['green','blue']

In [375]:
def cos_sim_recs(index, n=5, resort=None, color=None):
    trail = X[index].reshape(1,-1)
    cs = cosine_similarity(trail, X)
    rec_index = np.argsort(cs)[0][::-1][1:]
    ordered_df = df.loc[rec_index]
    if resort:
        ordered_df = ordered_df[ordered_df['resort'] == resort]
    if color:
        ordered_df = ordered_df[ordered_df['colors'].isin(color)]
    rec_df = ordered_df.head(n)
    rec_df = rec_df.reset_index(drop=True)
    rec_df.index = rec_df.index+1
    orig_row = df.loc[[index]].rename(lambda x: 'original')
    total = pd.concat((orig_row,rec_df))
    return total

In [378]:
cos_sim_recs(901,n=10,color=color)

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
original,Whistle Stop Lower,10800.0,10640.0,160.0,1876.64,79.0,4.74,9.0,13.0,Novice,Winter Park,CO,1,green,2,1
1,Upper White Rabbit,10800.0,10600.0,200.0,1902.59,124.0,4.52,11.0,16.0,Novice,Winter Park,CO,1,blue,2,2
2,Minor Matter,10677.0,10479.0,198.0,1723.00,100.0,4.00,12.0,18.0,Novice,Copper,CO,1,green,2,1
3,Gunbarrel,10670.0,10450.0,220.0,1991.16,50.0,4.00,11.0,12.0,Low Intermediate,Winter Park,CO,1,green,3,1
4,Peanut,10320.0,10165.0,156.0,1833.00,35.0,1.50,9.0,15.0,Novice,Crested Butte,CO,1,green,2,1
5,Mock Turtle,9965.0,9725.0,240.0,1915.39,110.0,5.02,13.0,16.0,Novice,Winter Park,CO,1,green,2,1
6,Silver Queen Road,11242.0,11074.0,168.0,2050.00,36.0,1.70,8.0,16.0,Low Intermediate,Crested Butte,CO,1,blue,3,2
7,Loop Road,7522.0,7381.0,141.0,1395.00,135.0,4.30,10.0,15.0,Intermediate,Alpine Meadows,CA,1,blue,4,2
8,Coyote Crossing,10220.0,10050.0,170.0,1559.30,35.0,1.30,11.0,12.0,Beginner,Vail,CO,1,green,1,1
9,Shoo Fly,9580.0,9450.0,130.0,1258.66,112.0,5.88,10.0,14.0,Beginner,Winter Park,CO,1,green,1,1


# Percent Change in recommendations

In [42]:
results_df = cos_sim_recommendations('Sorensen Park','Winter Park',X,n=5)

In [91]:
results_df

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
original,Sorensen Park,9150.0,9090.0,60.0,1067.43,114.0,4.07,6.0,8.0,Beginner,Winter Park,CO,1,green,1,1
821,School Yard,6799.0,6732.0,66.0,980.00,129.0,2.90,7.0,9.0,Beginner,Diamond Peak,NV,1,green,1,1
673,Bubba’s Shortcut Upper,10335.0,10238.0,97.0,1200.00,118.0,3.30,8.0,11.0,Novice,Crested Butte,CO,1,green,2,1
932,Shoo Fly,9580.0,9450.0,130.0,1258.66,112.0,5.88,10.0,14.0,Beginner,Winter Park,CO,1,green,1,1
314,Little Hawk TRV,9369.0,9348.0,21.0,821.00,67.0,1.30,3.0,8.0,Beginner,Eldora,CO,1,green,1,1
373,Loop Road,7522.0,7381.0,141.0,1395.00,135.0,4.30,10.0,15.0,Intermediate,Alpine Meadows,CA,1,blue,4,2


In [110]:
dfs = []
for num in range(1,6):
    dfs.append(results_df.iloc[[0,num]].select_dtypes(include=[np.number]))
pct_changes = []
for num in range(0,5):
    pct_changes.append(dfs[num].pct_change().iloc[[1]])
pct_change_df = pd.concat(pct_changes)
pct_change_df

,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),groomed,ability_nums,color_nums
821,-0.256940,-0.259406,0.100000,-0.081907,0.131579,-0.287469,0.166667,0.125,0.0,0.0,0.0
673,0.129508,0.126293,0.616667,0.124195,0.035088,-0.189189,0.333333,0.375,0.0,1.0,0.0
932,0.046995,0.039604,1.166667,0.179150,-0.017544,0.444717,0.666667,0.750,0.0,0.0,0.0
314,0.023934,0.028383,-0.650000,-0.230863,-0.412281,-0.680590,-0.500000,0.000,0.0,0.0,0.0
373,-0.177923,-0.188009,1.350000,0.306877,0.184211,0.056511,0.666667,0.875,0.0,3.0,1.0


In [113]:
pct_change_df['length'] = 'Longer'
pct_change_df['length'][pct_change_df['slope_length_(ft)'] == 0] = 'Same Length'
pct_change_df['length'][pct_change_df['slope_length_(ft)'] < 0] = 'Shorter'

In [115]:
pct_change_df['avg_slope'] = 'Steeper Overall'
pct_change_df['avg_slope'][pct_change_df['avg_grade_(%)'] == 0] = 'Same Average Slope'
pct_change_df['avg_slope'][pct_change_df['avg_grade_(%)'] < 0] = 'Less Steep Overall'

In [119]:
pct_change_df['max_slope'] = 'Steeper Max slope'
pct_change_df['max_slope'][pct_change_df['max_grade_(%)'] == 0] = 'Same Max Slope'
pct_change_df['max_slope'][pct_change_df['max_grade_(%)'] < 0] = 'Less Steep Max Slope'

In [121]:
pct_change_df['vert_rise'] = 'More Vertical'
pct_change_df['vert_rise'][pct_change_df['vert_rise_(ft)'] == 0] = 'Same Vertical'
pct_change_df['vert_rise'][pct_change_df['vert_rise_(ft)'] < 0] = 'Less Vertical'

In [122]:
pct_change_df['width'] = 'Wider Overall'
pct_change_df['width'][pct_change_df['avg_width_(ft)'] == 0] = 'Same Width Overall'
pct_change_df['width'][pct_change_df['avg_width_(ft)'] < 0] = 'Narrower Overall'

In [123]:
pct_change_df

,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),groomed,ability_nums,color_nums,length,avg_slope,max_slope,vert_rise,width
821,-0.256940,-0.259406,0.100000,-0.081907,0.131579,-0.287469,0.166667,0.125,0.0,0.0,0.0,Shorter,Steeper Overall,Steeper Max slope,More Vertical,Wider Overall
673,0.129508,0.126293,0.616667,0.124195,0.035088,-0.189189,0.333333,0.375,0.0,1.0,0.0,Longer,Steeper Overall,Steeper Max slope,More Vertical,Wider Overall
932,0.046995,0.039604,1.166667,0.179150,-0.017544,0.444717,0.666667,0.750,0.0,0.0,0.0,Longer,Steeper Overall,Steeper Max slope,More Vertical,Narrower Overall
314,0.023934,0.028383,-0.650000,-0.230863,-0.412281,-0.680590,-0.500000,0.000,0.0,0.0,0.0,Shorter,Less Steep Overall,Same Max Slope,Less Vertical,Narrower Overall
373,-0.177923,-0.188009,1.350000,0.306877,0.184211,0.056511,0.666667,0.875,0.0,3.0,1.0,Longer,Steeper Overall,Steeper Max slope,More Vertical,Wider Overall


## Trying out weighting features

In [16]:
X

array([[ 1.12885212,  0.67904826,  1.5011197 , ...,  1.7960483 ,
         1.3829293 , -0.81952402],
       [ 0.72015105,  0.79497328, -0.24838454, ...,  2.18280168,
         1.22823665, -0.81952402],
       [ 1.06270397,  1.1104155 , -0.15644362, ...,  1.1772429 ,
         1.74387879, -0.81952402],
       ..., 
       [-1.87458885, -1.49592585, -1.27286899, ..., -1.45268006,
        -1.6077951 ,  1.22022049],
       [-1.87222642, -1.49592585, -1.26498834, ..., -1.14327736,
        -1.55623089,  1.22022049],
       [-1.84545217, -1.49592585, -1.17567432, ..., -0.91122533,
        -1.55623089,  1.22022049]])

In [28]:
X[0]

array([ 1.12885212,  0.67904826,  1.5011197 ,  0.08264671,  0.27165507,
        0.3173219 ,  1.7960483 ,  1.3829293 , -0.81952402])

In [48]:
X_weighted_pt1 = X[:,:-3]
X_weighted_pt2 = X[:,-3]*2  # avg_grade
X_weighted_pt3 = X[:,-2]*2  # max_grade
X_weighted_pt4 = X[:,-1]
X_weighted = np.hstack((X_weighted_pt1,X_weighted_pt2.reshape(-1,1),X_weighted_pt3.reshape(-1,1),X_weighted_pt4.reshape(-1,1),))

In [33]:
X_weighted_pt1.shape

(1125, 6)

In [37]:
X_weighted = np.hstack((X_weighted_pt1,X_weighted_pt2.reshape(-1,1),X_weighted_pt3.reshape(-1,1),X_weighted_pt4.reshape(-1,1),))

In [38]:
X_weighted

array([[ 1.12885212,  0.67904826,  1.5011197 , ...,  3.5920966 ,
         2.76585859, -0.81952402],
       [ 0.72015105,  0.79497328, -0.24838454, ...,  4.36560335,
         2.45647331, -0.81952402],
       [ 1.06270397,  1.1104155 , -0.15644362, ...,  2.3544858 ,
         3.48775758, -0.81952402],
       ..., 
       [-1.87458885, -1.49592585, -1.27286899, ..., -2.90536012,
        -3.21559021,  1.22022049],
       [-1.87222642, -1.49592585, -1.26498834, ..., -2.28655471,
        -3.11246178,  1.22022049],
       [-1.84545217, -1.49592585, -1.17567432, ..., -1.82245066,
        -3.11246178,  1.22022049]])

In [67]:
w_results_df = cos_sim_recommendations('Jack Kendrick','Winter Park',X_weighted,n=10,resort='Winter Park')

In [68]:
w_results_df

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
original,Jack Kendrick,10135.0,9495.0,640.0,3851.74,140.0,11.78,17.0,26.0,Novice,Winter Park,CO,1,green,2,1
885,Low Lonesome Whistle,10420.0,9800.0,620.0,4382.38,96.0,10.18,14.0,24.0,Low Intermediate,Winter Park,CO,1,blue,3,2
975,Buckaroo,10220.0,9650.0,570.0,4041.95,154.0,14.33,14.0,27.0,Low Intermediate,Winter Park,CO,1,blue,3,2
879,Lower White Rabbit,10160.0,9720.0,440.0,2976.84,165.0,9.73,15.0,26.0,Novice,Winter Park,CO,1,green,2,1
896,Butch's Breezeway,10135.0,9490.0,645.0,3225.74,162.0,10.36,20.0,34.0,Low Intermediate,Winter Park,CO,1,blue,3,2
849,Turnpike,9460.0,9090.0,370.0,4366.51,87.0,5.15,9.0,15.0,Novice,Winter Park,CO,1,green,2,1
874,March Hare,10680.0,10140.0,540.0,2848.69,168.0,11.02,19.0,26.0,Novice,Winter Park,CO,1,green,2,1
964,Corona Way,10340.0,9580.0,760.0,7438.18,40.0,13.48,10.0,19.0,Low Intermediate,Winter Park,CO,1,blue,3,2
895,Easy Way,9750.0,9430.0,320.0,3942.64,100.0,3.39,8.0,8.0,Beginner,Winter Park,CO,1,green,1,1
968,Big Valley,9860.0,9560.0,300.0,3733.83,105.0,13.33,8.0,8.0,Low Intermediate,Winter Park,CO,1,green,3,1


In [69]:
results_df = cos_sim_recommendations('Jack Kendrick','Winter Park',X,n=10,resort='Winter Park')

In [70]:
results_df

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location,groomed,colors,ability_nums,color_nums
original,Jack Kendrick,10135.0,9495.0,640.0,3851.74,140.0,11.78,17.0,26.0,Novice,Winter Park,CO,1,green,2,1
975,Buckaroo,10220.0,9650.0,570.0,4041.95,154.0,14.33,14.0,27.0,Low Intermediate,Winter Park,CO,1,blue,3,2
885,Low Lonesome Whistle,10420.0,9800.0,620.0,4382.38,96.0,10.18,14.0,24.0,Low Intermediate,Winter Park,CO,1,blue,3,2
896,Butch's Breezeway,10135.0,9490.0,645.0,3225.74,162.0,10.36,20.0,34.0,Low Intermediate,Winter Park,CO,1,blue,3,2
873,Allen Phipps,10050.0,9470.0,580.0,3217.44,154.0,12.22,18.0,38.0,Low Intermediate,Winter Park,CO,1,green,3,1
879,Lower White Rabbit,10160.0,9720.0,440.0,2976.84,165.0,9.73,15.0,26.0,Novice,Winter Park,CO,1,green,2,1
849,Turnpike,9460.0,9090.0,370.0,4366.51,87.0,5.15,9.0,15.0,Novice,Winter Park,CO,1,green,2,1
964,Corona Way,10340.0,9580.0,760.0,7438.18,40.0,13.48,10.0,19.0,Low Intermediate,Winter Park,CO,1,blue,3,2
874,March Hare,10680.0,10140.0,540.0,2848.69,168.0,11.02,19.0,26.0,Novice,Winter Park,CO,1,green,2,1
895,Easy Way,9750.0,9430.0,320.0,3942.64,100.0,3.39,8.0,8.0,Beginner,Winter Park,CO,1,green,1,1
